<h1> Classifying news articles as business or political, based on their title </h1>

<h3> The goal of this notebook is to explore how one could use Tf-Idf and/or cosine similarity to classify news articles as business or political articles.</h3>

<p> The Notebook is structured in the following manner: </p>
<ol>
<li> Data Cleaning </li> 
<li> Data Analysis </li> 
<li> Model Building </li> 
<li> Model Experiments </li> 
</ol>

<h1 style="text-align:center"> Data cleaning </h1>

<p>Before we begin, we have to import all of our needed libraries. </p>

In [118]:
import numpy as np
import pandas as pd
import scipy as scp
import sklearn as sk
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter
import re
import string
import contractions


<p>Now that we're ready, let's load the data.</p>

In [119]:
newsDataset = pd.read_csv('./ArticleTitlesCategoryDataset.csv')
newsDataset.head()

,Title,Category,Source
0,"As Democrats try to hold on in November, it’s ...",Politics,CNN
1,A Dizzying week for Trump’s legal issues,Politics,CNN
2,Nancy Pelosi did what Donald Trump failed to d...,Politics,CNN
3,Donald Trump told us all *exactly* what he was...,Politics,CNN
4,See why Republicans are trying to get you to f...,Politics,CNN


<p>Alright, the dataset is loaded successfully, now let's clean the data a bit. We'll do the following: </p>
<ol>
    <li> Turn all the contraction words into proper words (don't -> do not, I'd -> I would). </li>
    <li> Remove all the punctuation from the title. </li>
    <li> Tokenize the title. </li>
</ol>

In [120]:
EnglishStopwords = set(stopwords.words('english'))

Titles = newsDataset['Title']
CleanedTitles = []
Lemmatizer = WordNetLemmatizer()
for title in Titles:
    NoContractionsTitle = contractions.fix(title)
    NoPunctuationTitle = NoContractionsTitle.translate(str.maketrans('','', string.punctuation)) 
    TokenizedTitle = word_tokenize(NoPunctuationTitle) 
    
    cleanTitle = []
    for word in TokenizedTitle:
        if word not in EnglishStopwords:
            LemmatizedVerb = Lemmatizer.lemmatize(word.lower(), pos='v')
            LemmatizedNoun = Lemmatizer.lemmatize(LemmatizedVerb, pos='n')
            cleanTitle.append(LemmatizedNoun)
    cleanTitle = " ".join(cleanTitle)
    
    CleanedTitles.append(cleanTitle)


In [121]:
CleanedDataset = pd.DataFrame(columns=['CleanTitle', 'Category', 'Source'])
CleanedDataset['CleanTitle'] = CleanedTitles
CleanedDataset['Category'] = newsDataset['Category']
CleanedDataset['Source'] = newsDataset['Source']

CleanedDataset.head(5)

,CleanTitle,Category,Source
0,a democrat try hold november pete buttigieg de...,Politics,CNN
1,a dizzy week trump ’ legal issue,Politics,CNN
2,nancy pelosi donald trump fail january 6,Politics,CNN
3,donald trump tell u exactly go november 2020,Politics,CNN
4,see republican try get focus wokeness,Politics,CNN


<p>Great ! We now have a clean titles dataset ! We can now proceed to the data analysis part!</p>

<h1 style="text-align:center"> Data Analysis </h1>

<p>Let's find the most common words in the titles of political articles. </p>
<p> First, we'll filter the data by the politics category. </p>

In [122]:
PoliticalArticles = CleanedDataset[CleanedDataset['Category'] == 'Politics']
PoliticalArticles.head(5)

,CleanTitle,Category,Source
0,a democrat try hold november pete buttigieg de...,Politics,CNN
1,a dizzy week trump ’ legal issue,Politics,CNN
2,nancy pelosi donald trump fail january 6,Politics,CNN
3,donald trump tell u exactly go november 2020,Politics,CNN
4,see republican try get focus wokeness,Politics,CNN


<p>Now that we have the political articles only, we can proceed with the extraction of the most common words in the titles. </p>

In [143]:
Titles = "".join(PoliticalArticles['CleanTitle'])
TitlesTokenized = word_tokenize(Titles)
fdist = FreqDist(TitlesTokenized)
fdist.most_common(5)

[('trump', 10),
 ('’', 8),
 ('november', 5),
 ('senate', 5),
 ('‘', 5),
 ('say', 5),
 ('democrat', 4),
 ('campaign', 4),
 ('u', 4),
 ('republican', 4),
 ('control', 4),
 ('warn', 4),
 ('biden', 4),
 ('election', 4),
 ('6', 4),
 ('gop', 4),
 ('tory', 4),
 ('try', 3),
 ('week', 3),
 ('pelosi', 3),
 ('january', 3),
 ('know', 3),
 ('call', 3),
 ('georgia', 3),
 ('midterm', 3),
 ('ask', 3),
 ('voter', 3),
 ('state', 3),
 ('mp', 3),
 ('conservative', 3),
 ('2024', 3),
 ('dizzy', 2),
 ('tell', 2),
 ('go', 2),
 ('focus', 2),
 ('may', 2),
 ('win', 2),
 ('abortion', 2),
 ('right', 2),
 ('event', 2),
 ('michigan', 2),
 ('agent', 2),
 ('latino', 2),
 ('secretary', 2),
 ('ye', 2),
 ('oil', 2),
 ('oust', 2),
 ('interest', 2),
 ('liz', 2),
 ('truss', 2),
 ('pm', 2),
 ('without', 2),
 ('watch', 2),
 ('christie', 2),
 ('argue', 2),
 ('dominate', 2),
 ('rather', 2),
 ('principle', 2),
 ('dems', 2),
 ('put', 2),
 ('warnock', 2),
 ('take', 2),
 ('claim', 2),
 ('probe', 2),
 ('request', 2),
 ('migrant', 2)